## Notebook setup

In [ ]:
import io
import math
import warnings
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
import pandas as pd
from tqdm.notebook import tqdm

from openbb_terminal import api as openbb
from openbb_terminal.helper_classes import TerminalStyle

%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
warnings.filterwarnings("ignore")

In [ ]:
try:
    theme = TerminalStyle("light", "light", "light")
except:
    pass
stylesheet = openbb.widgets.html_report_stylesheet()

## Select Ticker

In [ ]:
# Parameters that will be replaced when calling this notebook
ticker = "clx"
report_name = f"{datetime.now().strftime('%Y%m%d_%H%M%S')}_comparison_{ticker.upper()}"

In [ ]:
ticker = ticker.upper()
report_title = (
    f"{ticker.upper()} Comparison ({datetime.now().strftime('%Y-%m-%d %H:%M:%S')})"
)
report_title

In [ ]:
finviz_similar = openbb.stocks.ca.models.finviz_compare.get_similar_companies(
    ticker, ["Sector", "Industry", "Country"]
)[0]
finviz_similar.pop(finviz_similar.index(ticker))
print(f"Similar companies:\n{' '.join([i for i in finviz_similar])}")

### Overview

In [ ]:
overview = openbb.stocks.ca.models.finviz_compare.get_comparison_data(
    similar=finviz_similar,
    data_type="overview",
)
overview = overview.sort_values("Volume", ascending=False)
overview.head(4)

In [ ]:
# Update the list of similar companies to take up only up to 12 with top volume
finviz_similar = overview["Ticker"].to_list()[:12]
print(f"Similar companies:\n{' '.join([i for i in finviz_similar])}")

### Similar companies and descriptions

In [ ]:
def get_info_as_dict(ticker):
    df_info = openbb.stocks.fa.models.yahoo_finance.get_info(ticker)
    if "Long business summary" in df_info.index:
        summary = df_info.loc["Long business summary"].values[0]
    elif "Long name" in df_info.index:
        summary = df_info.loc["Long name"].values[0]
    else:
        summary = ""
    if "Website" in df_info.index:
        website = df_info.loc["Website"].values[0]
    else:
        website = ""
    return {"summary": summary, "website": website}


similar_companies = dict()
similar_companies[ticker] = get_info_as_dict(ticker)

if finviz_similar:
    for symbol in tqdm(finviz_similar):
        similar_companies[symbol] = get_info_as_dict(symbol)

### Historical prices

In [ ]:
for c in tqdm(finviz_similar):
    fig, ax = plt.subplots(figsize=(4.2, 3), dpi=150)
    openbb.stocks.ca.hist(
        normalize=True,
        similar_tickers=[ticker, c],
        external_axes=[ax],
    )
    ax.set_title("")
    fig.tight_layout()
    f = io.BytesIO()
    fig.savefig(f, format="svg")
    similar_companies[c]["historical_price"] = f.getvalue().decode("utf-8")

### Historical correlation

In [ ]:
fig, ax = plt.subplots(figsize=(9.5, 6), dpi=150)

openbb.stocks.ca.hcorr(
    similar_tickers=finviz_similar,
    external_axes=[ax],
)

fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
historical_correlation = f.getvalue().decode("utf-8")

### Historical volumes

In [ ]:
volume_groups = []
for i in range(math.ceil(len(finviz_similar) / 3)):
    volume_groups.append(finviz_similar[3 * (i) : 3 * (i + 1)])
volume_groups

In [ ]:
historical_volumes = []
for vgroup in volume_groups:
    fig, ax = plt.subplots(figsize=(4, 3), dpi=150)
    openbb.stocks.ca.volume(
        similar_tickers=vgroup,
        external_axes=[ax],
    )
    ax.set_title(" ".join([i for i in vgroup]))
    f = io.BytesIO()
    fig.savefig(f, format="svg")
    historical_volumes.append(f.getvalue().decode("utf-8"))

### Valuation

In [ ]:
valuation = openbb.stocks.ca.models.finviz_compare.get_comparison_data(
    similar=finviz_similar,
    data_type="valuation",
)
valuation.head(4)

### Financial

In [ ]:
financial = openbb.stocks.ca.models.finviz_compare.get_comparison_data(
    similar=finviz_similar,
    data_type="financial",
)
financial.head(4)

### Ownership

In [ ]:
ownership = openbb.stocks.ca.models.finviz_compare.get_comparison_data(
    similar=finviz_similar,
    data_type="ownership",
)
ownership.head(4)

### Performance

In [ ]:
performance = openbb.stocks.ca.models.finviz_compare.get_comparison_data(
    similar=finviz_similar,
    data_type="performance",
)
performance.head(4)

### Technical

In [ ]:
technical = openbb.stocks.ca.models.finviz_compare.get_comparison_data(
    similar=finviz_similar,
    data_type="technical",
)
technical.head(4)

## Render the report template to a file

In [ ]:
body = ""

In [ ]:
body += openbb.widgets.h(1, report_title)

body += openbb.widgets.h(2, "Overview")

for company in similar_companies:
    if company == ticker:
        continue
    # widgets.append()
    body += openbb.widgets.row(
        [
            openbb.widgets.h(3, company)
            + "<br>"
            + similar_companies[company]["website"]
            + "<br>"
            + "<br>"
            + similar_companies[company]["summary"],
            openbb.widgets.h(3, f"{ticker} vs {company}")
            + similar_companies[company]["historical_price"],
        ]
    )

body += openbb.widgets.h(2, "Historical correlation")
body += openbb.widgets.row([historical_correlation])

body += openbb.widgets.h(2, "Historical volumes")
for chart in range(0, len(historical_volumes), 2):
    body += openbb.widgets.row(
        [historical_volumes[chart], historical_volumes[chart + 1]]
    )


body += openbb.widgets.h(2, "Screeners")

body += openbb.widgets.h(3, "overview")
body += openbb.widgets.row([overview.to_html()])

body += openbb.widgets.h(3, "valuation")
body += openbb.widgets.row([valuation.to_html()])

body += openbb.widgets.h(3, "financial")
body += openbb.widgets.row([financial.to_html()])

body += openbb.widgets.h(3, "ownership")
body += openbb.widgets.row([ownership.to_html()])

body += openbb.widgets.h(3, "performance")
body += openbb.widgets.row([performance.to_html()])

In [ ]:
report = openbb.widgets.html_report(title=report_name, stylesheet=stylesheet, body=body)

In [ ]:
# to save the results
with open(report_name + ".html", "w") as fh:
    fh.write(report)